# Gets last data point for each stream per participant

In [1]:
import mysql.connector as mysql
from datetime import datetime
import time
import pandas as pd
import matplotlib.pyplot as plt
import json
import pdb

In [2]:
!pwd

/Users/mash2/Dropbox (PAC LAB)/codes/ionic_projects/SARAV3/SARA_modular/server_side_scripts/aware/data_checks


In [3]:
%%javascript
//toggle scrolling for jupyter notebooks.
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [4]:
def getSqlConfigFromJSON(configFileName):
    """
    Loads full mysql connect object from database returns the value as a JSON object. Format of the JSON object 
    is the following:
    
        {
            "host": "hostname.com",
            "port": 99999,
            "user": "root",
            "passwd": "passworkd",
            "database": "database_or_schema_name"
        }
        
    """
    
    with open(configFileName) as f:
        mysqlConnectObject = json.load(f)
        
    return mysqlConnectObject

## Unique users

In [9]:
def get_unique_users():
    """
    gets unique users from database
    """
    
    #get sql connect config
    mysql_connect_object = getSqlConfigFromJSON('./awareSqlConfig.json')

    # connect to db
    db = mysql.connect(
        host = mysql_connect_object["host"],
        port = mysql_connect_object["port"],
        user = mysql_connect_object["user"],
        passwd = mysql_connect_object["passwd"],
        database = mysql_connect_object["database"]
    )
    cursor = db.cursor()

    # fetch data
    cursor.execute("SELECT * FROM aware_device;")
    record_in_user_table = cursor.fetchall()
    
    # fetch unique ids
    unique_aware_ids = []
    for row in record_in_user_table:
        unique_aware_ids.append(row[2])
        
    return unique_aware_ids
        
unique_aware_ids = get_unique_users()

print('Unique ids are')
unique_aware_ids

Unique ids are


['36c38a8d-c6d7-48a7-88a5-5f9796b540b6',
 '7134c8fb-07e9-4075-b3a9-8a0443f9d7a7',
 '6f0273ed-0b9b-4188-a008-39dfda32247e',
 '38ce91a4-0cb5-43bf-9e35-60da0202662a',
 'b39a842a-9a42-4ef0-aa3f-8a9c2ff6c018']

## Get latest data

Todo: 
- Convert date to readable timestamp value
- Get users timezone

In [17]:
def get_database_object():
    #get sql connect config
    mysqlConnectObject = getSqlConfigFromJSON('./awareSqlConfig.json')

    # connect to db
    db = mysql.connect(
        host = mysqlConnectObject["host"],
        port = mysqlConnectObject["port"],
        user = mysqlConnectObject["user"],
        passwd = mysqlConnectObject["passwd"],
        database = mysqlConnectObject["database"]
    )
    
    return db

def get_latest_for_user(aware_id, table_name, db):
    """
    Returns the latest time for aware_id for the given table.
    Time is returned in utc unix time stamp
    """    
    cursor = db.cursor()

    # fetch data.
    cursor.execute("SELECT * FROM " + table_name + " where device_id='" + aware_id + "' order by timestamp desc limit 1;")
    row = cursor.fetchone()
    return row[1]
    
tables = ['screen', 'locations']

db = get_database_object()
latest_time = {}
for aware_id in unique_aware_ids:
    for table in tables:
        if aware_id not in latest_time:
            latest_time[aware_id] = {}
            
        ts = get_latest_for_user(aware_id, table, db)
        ts = ts/1000        
        datetime_ts = datetime.utcfromtimestamp(ts) 
        latest_time[aware_id][table] = datetime_ts.strftime('%Y-%m-%d %I:%M:%S %p') + " UTC"
        

print(json.dumps(latest_time, indent=4, sort_keys=True))

{
    "36c38a8d-c6d7-48a7-88a5-5f9796b540b6": {
        "locations": "2021-05-17 11:35:58 PM UTC",
        "screen": "2021-05-17 09:38:25 PM UTC"
    },
    "38ce91a4-0cb5-43bf-9e35-60da0202662a": {
        "locations": "2021-05-18 07:12:45 AM UTC",
        "screen": "2021-05-18 04:12:57 AM UTC"
    },
    "6f0273ed-0b9b-4188-a008-39dfda32247e": {
        "locations": "2021-05-18 02:53:25 AM UTC",
        "screen": "2021-05-18 02:40:39 AM UTC"
    },
    "7134c8fb-07e9-4075-b3a9-8a0443f9d7a7": {
        "locations": "2021-05-04 06:24:16 AM UTC",
        "screen": "2021-05-03 04:50:15 AM UTC"
    },
    "b39a842a-9a42-4ef0-aa3f-8a9c2ff6c018": {
        "locations": "2021-05-18 04:08:27 AM UTC",
        "screen": "2021-05-18 02:08:02 AM UTC"
    }
}
